# Retrain Final Model & Evaluation

In [9]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [2]:
# Load retrain data
train_df = pd.read_parquet('../data/retrain/final_train.parquet')
test_df = pd.read_parquet('../data/retrain/final_test.parquet')

In [3]:
# Prepare data for lightgbm
X_train, Y_train = train_df.drop(columns=['loss', 'log_loss']), train_df['log_loss']

train_lgb = lgb.Dataset(X_train, label=Y_train)

params = {
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 404,
    'max_depth': 8, 
    'min_data_in_leaf': 202, 
    'learning_rate': 0.02361306858113298, 
    'feature_fraction': 0.7211579800007945, 
    'feature_fraction_bynode': 0.9081132434036039, 
    'bagging_fraction': 0.8727967232517579, 
    'bagging_freq': 7, 
    'lambda_l1': 5.763583420124947, 
    'lambda_l2': 14.460054311753241, 
    'min_gain_to_split': 0.03291221t8537174215, 
    'max_bin': 127,
    'boosting_type': 'gbdt',
    'seed': 42,
}

In [4]:
# Train model
model = lgb.train(
    params,
    train_lgb,
    num_boost_round=1400   # Use best_iterations for this set of params during tuning phase
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.749579 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 45262
[LightGBM] [Info] Number of data points in the train set: 169486, number of used features: 4843
[LightGBM] [Info] Start training from score 7.685626
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [5]:
test_df

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,log_cont12,cont12_rank,cont12_cap,log_cont13,cont13_rank,cont13_cap,log_cont14,cont14_rank,cont14_cap,log_loss
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.489103,0.506135,0.630853,0.557610,0.671388,0.746494,0.472007,0.624918,0.603209,7.434435
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.535484,0.570552,0.708275,0.509067,0.589235,0.663739,0.585141,0.872335,0.795244,7.452971
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.564679,0.610429,0.758883,0.499988,0.575071,0.648701,0.498307,0.663443,0.645933,7.938831
3,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.366983,0.361963,0.443374,0.606431,0.781870,0.833874,0.602497,0.956474,0.826675,6.331217
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.454967,0.463190,0.576121,0.503652,0.580737,0.654753,0.174446,0.015773,0.194667,6.165775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18826,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.556548,0.598160,0.744640,0.560477,0.677054,0.751507,0.516476,0.680809,0.676111,6.627949
18827,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.443005,0.447853,0.557380,0.581759,0.719547,0.789182,0.206540,0.086228,0.229417,8.690883
18828,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.251598,0.223926,0.286079,0.274413,0.291785,0.315758,0.219974,0.129699,0.246044,8.388732
18829,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.457327,0.466258,0.579845,0.614174,0.804533,0.848129,0.608174,0.981644,0.837073,8.061673


## Evaluation on test split

In [10]:
loss_true = test_df['loss']
loss_pred = np.exp(model.predict(test_df.drop(columns=['loss','log_loss'])))
mae = mean_absolute_error(loss_true, loss_pred)
print('MAE : ', mae)

MAE :  1124.2734102480895


In [16]:
# Export model
model.save_model('final_model.txt')